In [1]:
!pip install streamlit
!pip install streamlit pandas scikit-learn matplotlib seaborn
!pip install streamlit pandas numpy Pillow
!pip install streamlit plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
%%writefile app.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
import os

# Título do aplicativo
st.title('Detecção de Fraudes em Transações Financeiras')

# Caminho do arquivo CSV no Google Drive
file_path = '/content/drive/My Drive/Project/creditcard.csv'

# Verificando o caminho do arquivo
if os.path.exists(file_path):
    # Carregando os dados do arquivo CSV
    data = pd.read_csv(file_path)
    st.write("Dados carregados com sucesso.")
    st.write(data.head())  # Mostrar os primeiros registros para verificação
else:
    st.error(f"O arquivo em {file_path} não existe.")
    st.stop()

# Visualização dos dados
st.subheader('Visualização dos Dados')
st.write(data.head())

# Verificar as colunas disponíveis no DataFrame
st.subheader('Colunas Disponíveis')
st.write(data.columns)

# Determinar a coluna de fraudes
fraud_column = st.selectbox('Selecione a coluna que indica fraudes', data.columns)

# Gráfico de barras para a distribuição da variável target
st.subheader('Distribuição de Transações Fraudulentas e Não Fraudulentas')
fig, ax = plt.subplots()
sns.countplot(x=fraud_column, data=data, ax=ax)
ax.set_title('Distribuição de Transações Fraudulentas e Não Fraudulentas')
ax.set_xlabel('Fraude')
ax.set_ylabel('Contagem')
st.pyplot(fig)

# Gráfico de dispersão para visualização das features
st.subheader('Visualização das Features')
selected_x_feature = st.selectbox('Selecione a feature para o eixo X', data.columns)
selected_y_feature = st.selectbox('Selecione a feature para o eixo Y', data.columns)
fig, ax = plt.subplots()
sns.scatterplot(x=selected_x_feature, y=selected_y_feature, hue=fraud_column, data=data, ax=ax)
ax.set_title('Gráfico de Dispersão das Features')
st.pyplot(fig)

# Pré-processamento dos dados (exemplo)
features = st.multiselect('Selecione as features', data.columns)
if fraud_column in data.columns:
    y = data[fraud_column]
else:
    st.error(f"A coluna '{fraud_column}' não está presente nos dados.")
    st.stop()

X = data[features]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Escolha do algoritmo
classifier = st.selectbox('Escolha o algoritmo', ('SVM', 'Regressão Logística', 'Árvore de Decisão'))

# Treinamento do modelo
if classifier == 'SVM':
    model = SVC()
elif classifier == 'Regressão Logística':
    model = LogisticRegression()
else:
    model = DecisionTreeClassifier()
model.fit(X_train_scaled, y_train)

# Avaliação do modelo
st.subheader('Avaliação do Modelo')
y_pred = model.predict(X_test_scaled)
st.write('Matriz de Confusão:')
st.write(confusion_matrix(y_test, y_pred))
st.write('Relatório de Classificação:')
st.write(classification_report(y_test, y_pred))

# Gráfico da matriz de confusão
st.subheader('Gráfico da Matriz de Confusão')
fig, ax = plt.subplots()
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues', ax=ax)
ax.set_title('Matriz de Confusão')
ax.set_xlabel('Previsão')
ax.set_ylabel('Verdadeiro')
st.pyplot(fig)

# Detecção de Fraudes em Nova Transação
st.subheader('Detecção de Fraudes em Nova Transação')
new_transaction = {}  # Dicionário para armazenar os valores da nova transação
for feature in features:
    new_transaction[feature] = st.number_input(f'Insira o valor de {feature}', format="%.2f")
if new_transaction:
    new_transaction_df = pd.DataFrame([new_transaction])
    new_transaction_scaled = scaler.transform(new_transaction_df)
    prediction = model.predict(new_transaction_scaled)
    st.write('Previsão:', 'Fraude' if prediction[0] == 1 else 'Não Fraude')


Overwriting app.py


In [9]:
! ls -la # verifica arquivos
! cat app.py # visualizar conteudo do arquivo

total 40
drwxr-xr-x  1 root root 4096 Jun 10 12:53 .
drwxr-xr-x  1 root root 4096 Jun 10 12:50 ..
-rw-r--r--  1 root root 4064 Jun 10 13:47 app.py
drwxr-xr-x  4 root root 4096 Jun  6 14:15 .config
drwx------  5 root root 4096 Jun 10 12:53 drive
-rw-r--r--  1 root root 2372 Jun 10 13:00 logs.txt
drwxr-xr-x 25 root root 4096 Jun 10 12:53 node_modules
-rw-r--r--  1 root root 6709 Jun 10 12:53 package-lock.json
drwxr-xr-x  1 root root 4096 Jun  6 14:21 sample_data
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
import os

# Título do aplicativo
st.title('Detecção de Fraudes em Transações Financeiras')

# Caminho do arquivo CSV no Google Drive
file_p

In [10]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.574s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [1]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.143.144.50
npx: installed 22 in 3.707s
your url is: https://cold-goats-rule.loca.lt
